zebra puzzles

In [7]:
from copy import deepcopy
from random import randint
from icecream import ic
from tqdm.auto import tqdm
from random import choice, random

In [8]:
NUM_ITEM = 50
NUM_CATEGORIES = 10


In [9]:
ITEMS = [[(c,n) for n in range(NUM_ITEM)] for c in range(NUM_CATEGORIES)]
INCOMPATIBLE = [{randint(0,NUM_ITEM-1),randint(0,NUM_ITEM-1) , (randint(0,NUM_CATEGORIES-1),randint(0,NUM_CATEGORIES-1))} for _ in range(10000)]


In [10]:
def invalid(bag: set) -> bool:
    return any(i <= bag for i in INCOMPATIBLE)
def cost(solution: list[set]) -> int:
    return sum(invalid(b) for b in solution)

In [ ]:


#1.all mutation is possible
#2.mutation is not too big -- small change
#3.expolration vs exploitation -- different phase of the search
def tweak(bag: list[set],t:float = 0.1) -> list[set]:
    new_bags = deepcopy(bag)

    again = True
    while again:
        # b1 = choice(i for i in range(NUM_ITEM) if check(new_bags[i]))
        candidates = [i for i in range(NUM_ITEM) if invalid(new_bags[i])]
        if not candidates:
            # 没有任何满足 check 的袋子，直接返回或打破循环
            return new_bags  # 或者：break

        b1 = choice(candidates)
        b2 = randint(0,NUM_ITEM-1)
        b1_,b2_ = sorted(new_bags[b1]),sorted(new_bags[b2])
        i = randint(0,NUM_CATEGORIES-1)
        b1_[i],b2_[i] = b2_[i],b1_[i]
        new_bags[b1],new_bags[b2] = set(b1_),set(b2_)
        again = random() < t
    return new_bags

In [12]:
current_solution = [{ITEMS[c][i] for c in range(NUM_CATEGORIES)} for i in range(NUM_ITEM)]
current_cost = cost(current_solution)

ic(current_cost)

max_steps = 100

for steps in tqdm(range(1000)):
    if current_cost == 0:
        break
    new_solution = tweak(current_solution,t=0.2)
    new_cost = cost(new_solution)
    if new_cost < current_cost:
        current_solution = new_solution
        current_cost = new_cost 
        ic(current_cost)
print(current_cost)

ic| current_cost: 0
  0%|          | 0/1000 [00:00<?, ?it/s]

0
